# n_ai problem eda

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.cross_validation import train_test_split
%matplotlib inline
plt.style.use('ggplot')

D:\soft\dev\anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [3]:
random_state = 42
data = pd.read_csv('./../data/nai_train.csv')
data.head(2)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,target
0,0.263669,0.711012,0.730641,0.259069,0.331150,0.300947,0.468118,0.426362,0.545662,0.472496,...,0.869772,0.219606,0.690553,0.697929,0.332535,0.500681,0.418926,0.272475,0.822392,1
1,0.828056,0.433518,0.435617,0.209394,0.508133,0.296114,0.082211,0.876144,0.201528,0.630515,...,0.570470,0.235018,0.588986,0.767599,0.585097,0.458801,0.960031,0.732236,0.543159,0


In [4]:
X = data.drop(['target'], axis=1)
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=random_state)

In [5]:
from sklearn.grid_search import GridSearchCV

def grid_search_clf(clf_to_train, param_grid, X_train, y_train):
    clf = GridSearchCV(estimator=clf_to_train, param_grid=param_grid,
                   n_jobs=-1, scoring='log_loss', verbose=True)
    clf.fit(X_train, y_train)
    print('best score: {0}'.format(clf.best_score_))
    print('best estimator:')
    print(clf.best_estimator_)
    return clf

def clf_score(clf, X_test, y_test):
    clf_probs = clf.predict_proba(X_test)
    return log_loss(y_test, clf_probs)

In [6]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(random_state=random_state)

param_grid = {
    'n_estimators': [i for i in range(175, 186)],
#     'criterion': ['gini', 'entropy'],
#     'max_features': ['sqrt', 'log2']
#     'max_depth': [i for i in range(2, 16)]#,
    'min_samples_split': [1, 3, 6],
    'min_samples_leaf': [1, 3, 6]
    
}

clf = grid_search_clf(clf_rf, param_grid, X_train, y_train)
print('log_loss: {0}'.format(clf_score(clf, X_test, y_test)))

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 17.0min
[Parallel(n_jobs=-1)]: Done 297 out of 297 | elapsed: 25.7min finished


Fitting 3 folds for each of 99 candidates, totalling 297 fits
best score: -0.694912146854
best estimator:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=6, min_samples_split=1,
            min_weight_fraction_leaf=0.0, n_estimators=181, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
log_loss: 0.695207917162


In [40]:
# log_loss: 0.69872587194715363
# LOGLOSS 0.69855 on real data
# RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, n_estimators=180, n_jobs=1,
#            oob_score=False, random_state=42, verbose=0, warm_start=False)

# log_loss: 0.695207917162
# LOGLOSS 0.69826 on real data
# RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#             max_depth=None, max_features='auto', max_leaf_nodes=None,
#             min_samples_leaf=6, min_samples_split=1,
#             min_weight_fraction_leaf=0.0, n_estimators=181, n_jobs=1,
#             oob_score=False, random_state=42, verbose=0, warm_start=False)

In [11]:
def get_ans(clf, X, y, test_data):
    X_real_test = test_data.drop(['t_id'], axis=1)
    clf.fit(X, y)
    real_pred = clf.predict_proba(X_real_test)
    pred = [it[1] for it in real_pred]
    ans = pd.DataFrame(test_data['t_id'])
    ans['probability'] = pred
    return ans

In [8]:
test_data = pd.read_csv('./../data/nai_test.csv')
test_data.head(2)

,t_id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21
0,8309,0.853061,0.991828,0.350143,0.654192,0.909093,0.862609,0.463147,0.911227,0.406167,...,0.925065,0.913353,0.653512,0.962262,0.951630,0.886714,0.352522,0.108802,0.945558,0.007984
1,9017,0.966091,0.725498,0.560994,0.951211,0.135138,0.932749,0.711090,0.762113,0.021480,...,0.784351,0.456816,0.823934,0.225814,0.391988,0.058691,0.431277,0.403834,0.130560,0.922161


In [12]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=6, min_samples_split=1,
            min_weight_fraction_leaf=0.0, n_estimators=181, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

ans = get_ans(clf, X, y, test_data)

In [14]:
ans.to_csv('test_ans.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(random_state=random_state)

param_grid = {
    'n_estimators': [i for i in range(175, 186)],
#     'criterion': ['gini', 'entropy'],
#     'max_features': ['sqrt', 'log2']
#     'max_depth': [i for i in range(2, 16)]#,
    'min_samples_split': [1, 3, 6],
    'min_samples_leaf': [1, 3, 6]
    
}

clf = grid_search_clf(clf_rf, param_grid, X_train, y_train)
print('log_loss: {0}'.format(clf_score(clf, X_test, y_test)))